In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  27358      0 --:--:-- --:--:-- --:--:-- 27358
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
     |████████████████████████████████| 61kB 3.2MB/s 
Uninstalling torch-1.7.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
  Successfully uninstalled torch-1.7.0+cu101
Uninstalling torchvision-0.8.1+cu101:
  Successfully uninstalled torchvision-0.8.1+cu101
Done updating TPU runtime
Copying gs://tpu-pytorch/wheels/torch-nightly-cp37-cp37m-linux_x86_64.whl...
|
Operation completed over 1 objects/122.6 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp37-cp37m-linux_x86_64.whl...
\ [1 files]

In [ ]:
!unzip drive/MyDrive/highly-probable/imgs-300w.zip 

Streaming output truncated to the last 5000 lines.
  inflating: imgs-300w/15464274_2.jpg  
  inflating: imgs-300w/16233769_5.jpg  
  inflating: imgs-300w/15093270_1.jpg  
  inflating: imgs-300w/12404917_2.jpg  
  inflating: imgs-300w/15959393_3.jpg  
  inflating: imgs-300w/15733294_2.jpg  
  inflating: imgs-300w/16181030_1.jpg  
  inflating: imgs-300w/16340654_6.jpg  
  inflating: imgs-300w/14708964_2.jpg  
  inflating: imgs-300w/16011282_2.jpg  
  inflating: imgs-300w/14968364_4.jpg  
  inflating: imgs-300w/16409913_2.jpg  
  inflating: imgs-300w/16367046_2.jpg  
  inflating: imgs-300w/16274128_4.jpg  
  inflating: imgs-300w/15705283_5.jpg  
  inflating: imgs-300w/16337843_1.jpg  
  inflating: imgs-300w/14397406_4.jpg  
  inflating: imgs-300w/13414118_2.jpg  
  inflating: imgs-300w/15796297_4.jpg  
  inflating: imgs-300w/16245478_3.jpg  
  inflating: imgs-300w/16114649_8.jpg  
  inflating: imgs-300w/16027830_5.jpg  
  inflating: imgs-300w/15841774_3.jpg  
  inflating: imgs-300w/163839

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp37-none-any.whl size=16031 sha256=7bafc94f0fde16ed5ad81aa1cf8266e1b5c60e164504fa858451f1481341b55d
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [ ]:
import gc
import os
import torch
import albumentations

import numpy as np
import pandas as pd

import torch.nn as nn
from sklearn import metrics
from sklearn import model_selection
from torch.nn import functional as F

# from wtfml.engine import Engine
# from wtfml.utils import EarlyStopping
# from wtfml.data_loaders.image import ClassificationDataLoader


import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

import efficientnet_pytorch

In [ ]:
NUM_EPOCHS = 15

In [ ]:
class EfficientNet(nn.Module):
    def __init__(self):
        super(EfficientNet, self).__init__()
        self.base_model = efficientnet_pytorch.EfficientNet.from_pretrained(
            'efficientnet-b0'
        )
        self.base_model._fc = nn.Linear(
            in_features=1280, 
            out_features=23, 
            bias=True
        )
        
    def forward(self, image, targets):
        out = self.base_model(image)
        #loss = nn.BCEWithLogitsLoss()(out, targets.view(-1, 1).type_as(out))
        loss = nn.CrossEntropyLoss()(out, targets.type_as(out))

        accuracy = metrics.accuracy_score(targets.cpu(), torch.argmax(out.cpu(), dim=1))

        return out, loss, accuracy

In [ ]:
DATA_DIR = 'imgs-300w'

def load_df():
    df = pd.read_csv("drive/MyDrive/highly-probable/womens-clothing-imageid-5fold.csv")
    # label encoding
    df['target'] = df['mid_category'].astype('category').cat.codes

    # remove rows with missing images (9 images)
    df['image_exists'] = df['image_name'].apply(lambda x: os.path.exists(f'{DATA_DIR}/{x}'))
    df = df[df['image_exists']]

    df = df[['image_name', 'target', 'fold']]

    df = df.rename(columns={'fold':'kfold'})

    
    print(df.shape)

    return df

df = load_df()
df.head()

(277620, 3)


,image_name,target,kfold
0,16222141_1.jpg,6,0.0
1,16222141_2.jpg,6,0.0
2,16222141_3.jpg,6,0.0
3,16222141_4.jpg,6,0.0
4,16222141_5.jpg,6,0.0


In [ ]:
MX = EfficientNet()

Loaded pretrained weights for efficientnet-b0


In [ ]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [ ]:
try:
    import torch_xla.core.xla_model as xm

    _xla_available = True
except ImportError:
    _xla_available = False


class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.0001, tpu=False):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.tpu = tpu
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

        if self.tpu and not _xla_available:
            raise Exception(
                "You want to use TPUs but you dont have pytorch_xla installed"
            )

    def __call__(self, epoch_score, model, model_path):
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.tpu:
                xm.master_print(
                    "EarlyStopping counter: {} out of {}".format(
                        self.counter, self.patience
                    )
                )
            else:
                print(
                    "EarlyStopping counter: {} out of {}".format(
                        self.counter, self.patience
                    )
                )
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            if self.tpu:
                xm.master_print(
                    "Validation score improved ({} --> {}). Saving model!".format(
                        self.val_score, epoch_score
                    )
                )
            else:
                print(
                    "Validation score improved ({} --> {}). Saving model!".format(
                        self.val_score, epoch_score
                    )
                )
            if self.tpu:
                xm.save(model.state_dict(), model_path)
            else:
                torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [ ]:
import cv2
import torch

import numpy as np

from PIL import Image
from PIL import ImageFile

try:
    import torch_xla.core.xla_model as xm

    _xla_available = True
except ImportError:
    _xla_available = False

ImageFile.LOAD_TRUNCATED_IMAGES = True


class ClassificationDataset:
    def __init__(
        self,
        image_paths,
        targets,
        resize,
        augmentations=None,
        backend="pil",
        channel_first=True,
    ):
        """
        :param image_paths: list of paths to images
        :param targets: numpy array
        :param resize: tuple or None
        :param augmentations: albumentations augmentations
        """
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
        self.backend = backend
        self.channel_first = channel_first

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        targets = self.targets[item]
        if self.backend == "pil":
            image = Image.open(self.image_paths[item])
            if self.resize is not None:
                image = image.resize(
                    (self.resize[1], self.resize[0]), resample=Image.BILINEAR
                )
            image = np.array(image)
            if self.augmentations is not None:
                augmented = self.augmentations(image=image)
                image = augmented["image"]
        elif self.backend == "cv2":
            image = cv2.imread(self.image_paths[item])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.resize is not None:
                image = cv2.resize(
                    image,
                    (self.resize[1], self.resize[0]),
                    interpolation=cv2.INTER_CUBIC,
                )
            if self.augmentations is not None:
                augmented = self.augmentations(image=image)
            image = augmented["image"]
        else:
            raise Exception("Backend not implemented")
        if self.channel_first:
            image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image),
            "targets": torch.tensor(targets),
        }


class ClassificationDataLoader:
    def __init__(
        self,
        image_paths,
        targets,
        resize,
        augmentations=None,
        backend="pil",
        channel_first=True,
    ):
        """
        :param image_paths: list of paths to images
        :param targets: numpy array
        :param resize: tuple or None
        :param augmentations: albumentations augmentations
        """
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
        self.backend = backend
        self.channel_first = channel_first
        self.dataset = ClassificationDataset(
            image_paths=self.image_paths,
            targets=self.targets,
            resize=self.resize,
            augmentations=self.augmentations,
            backend=self.backend,
            channel_first=self.channel_first,
        )

    def fetch(
        self,
        batch_size,
        num_workers,
        drop_last=False,
        shuffle=True,
        tpu=False,
        sampler=None,
    ):
        """
        :param batch_size: batch size
        :param num_workers: number of processes to use
        :param drop_last: drop the last batch?
        :param shuffle: True/False
        :param tpu: True/False, to use tpu or not
        """
        if tpu:
            sampler = torch.utils.data.distributed.DistributedSampler(
                self.dataset,
                num_replicas=xm.xrt_world_size(),
                rank=xm.get_ordinal(),
                shuffle=shuffle,
            )

        data_loader = torch.utils.data.DataLoader(
            self.dataset,
            batch_size=batch_size,
            sampler=sampler,
            drop_last=drop_last,
            num_workers=num_workers,
        )
        return data_loader

In [ ]:
import datetime
import torch
from tqdm import tqdm
 
try:
    from torch.cuda import amp
    _amp_available = True
except ImportError:
    _amp_available = False
 
try:
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.parallel_loader as pl
 
    _xla_available = True
except ImportError:
    _xla_available = False
 
 
def reduce_fn(vals):
    return sum(vals) / len(vals)
 
 
class Engine:
    def __init__(
        self,
        model,
        optimizer,
        device,
        scheduler=None,
        accumulation_steps=1,
        use_tpu=False,
        tpu_print=10,
        fp16=False,
        model_fn=None,
        use_mean_loss=False,
    ):
        """
        model_fn should take batch of data, device and model and return loss
        for example:
            def model_fn(data, device, model):
                images, targets = data
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                _, loss = model(images, targets)
                return loss
        """
        self.model = model
        self.optimizer = optimizer
        self.device = device
        self.scheduler = scheduler
        self.accumulation_steps = accumulation_steps
        self.use_tpu = use_tpu
        self.tpu_print = tpu_print
        self.model_fn = model_fn
        self.fp16 = fp16
        if self.fp16 and not _amp_available:
            raise Exception(
                "You want to use fp16 but dont have amp installed"
            )
        self.use_mean_loss = use_mean_loss
        self.scaler = None
 
        if self.use_tpu and not _xla_available:
            raise Exception(
                "You want to use TPUs but you dont have pytorch_xla installed"
            )
        if self.fp16 and use_tpu:
            raise Exception("Apex fp16 is not available when using TPUs")
        if self.fp16:
            self.scaler = amp.GradScaler()
 
    def train(self, data_loader):
        losses = AverageMeter()
        self.model.train()
        print_idx = int(len(data_loader) * self.tpu_print / 100)
        if self.accumulation_steps > 1:
            self.optimizer.zero_grad()
        if self.use_tpu:
            para_loader = pl.ParallelLoader(data_loader, [self.device])
            tk0 = para_loader.per_device_loader(self.device)
        else:
            tk0 = tqdm(data_loader, total=len(data_loader))
 
        for b_idx, data in enumerate(tk0):
            if self.accumulation_steps == 1 and b_idx == 0:
                self.optimizer.zero_grad()
 
            if self.model_fn is None:
                for key, value in data.items():
                    data[key] = value.to(self.device)
                _, loss = self.model(**data)
            else:
                if self.fp16:
                    with amp.autocast():
                        loss = self.model_fn(data, self.device, self.model)
                else:
                    loss = self.model_fn(data, self.device, self.model)
 
            if not self.use_tpu:
                with torch.set_grad_enabled(True):
                    if self.use_mean_loss:
                        loss = loss.mean()
 
                    if self.fp16:
                        self.scaler.scale(loss).backward()
                    else:
                        loss.backward()
 
                    if (b_idx + 1) % self.accumulation_steps == 0:
                        if self.fp16:
                            self.scaler.step(self.optimizer)
                        else:
                            self.optimizer.step()
 
                        if self.scheduler is not None:
                            self.scheduler.step()
 
                        if self.fp16:
                            self.scaler.update()
 
                        if b_idx > 0:
                            self.optimizer.zero_grad()
            else:
                loss.backward()
                xm.optimizer_step(self.optimizer)
                if self.scheduler is not None:
                    self.scheduler.step()
                if b_idx > 0:
                    self.optimizer.zero_grad()
            if self.use_tpu:
                reduced_loss = xm.mesh_reduce("loss_reduce", loss, reduce_fn)
                losses.update(reduced_loss.item(), data_loader.batch_size)
                

            else:
                losses.update(loss.item(), data_loader.batch_size)
 
            if not self.use_tpu:
                tk0.set_postfix(loss=losses.avg)
            else:
                # if b_idx % print_idx == 0 or b_idx == len(data_loader):
                if b_idx % 10 == 0 or b_idx == len(data_loader):
                    xm.master_print(
                        f"{datetime.datetime.now()}: Batch {b_idx} / {len(data_loader)}, loss={losses.avg}"
                    )
        if not self.use_tpu:
            tk0.close()
        return losses.avg
 
    def evaluate(self, data_loader, return_predictions=False):
        losses = AverageMeter()
        accs = AverageMeter()
        print_idx = int(len(data_loader) * self.tpu_print / 100)
        self.model.eval()
        final_predictions = []
        with torch.no_grad():
            if self.use_tpu:
                para_loader = pl.ParallelLoader(data_loader, [self.device])
                tk0 = para_loader.per_device_loader(self.device)
            else:
                tk0 = tqdm(data_loader, total=len(data_loader))
            for b_idx, data in enumerate(tk0):
                for key, value in data.items():
                    data[key] = value.to(self.device)
                if self.fp16:
                    with amp.autocast():
                        batch_preds, loss, accuracy = self.model(**data)
                else:
                    batch_preds, loss, accuracy = self.model(**data)
                if return_predictions:
                    final_predictions.append(batch_preds)
                if self.use_tpu:
                    reduced_loss = xm.mesh_reduce("loss_reduce", loss, reduce_fn)
                    losses.update(reduced_loss.item(), data_loader.batch_size)
                    
                    reduced_accuracy = xm.mesh_reduce("acc_reduce", accuracy, reduce_fn)
                    accs.update(reduced_accuracy.item(), data_loader.batch_size)
                else:
                    if self.use_mean_loss:
                        loss = loss.mean()
                    losses.update(loss.item(), data_loader.batch_size)
                if not self.use_tpu:
                    tk0.set_postfix(loss=losses.avg)
                else:
                    #if b_idx % print_idx == 0 or b_idx == len(data_loader):
                    if b_idx % 10 == 0 or b_idx == len(data_loader):
                        xm.master_print(
                            f"{datetime.datetime.now()}: Batch {b_idx} / {len(data_loader)}, loss={losses.avg}"
                        )
            if not self.use_tpu:
                tk0.close()
        return accs.avg, losses.avg, final_predictions
 
    def predict(self, data_loader):
        self.model.eval()
        final_predictions = []
        if self.use_tpu:
            raise Exception("TPU not available for predict yet!")
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader))
            for data in tk0:
                for key, value in data.items():
                    data[key] = value.to(self.device)
                predictions, _ = self.model(**data)
                predictions = predictions.cpu()
                final_predictions.append(predictions)
        return final_predictions

In [ ]:
def train():
    training_data_path = "imgs-300w/"
    df = load_df()
    device = xm.xla_device()
    epochs = NUM_EPOCHS
    train_bs = 32
    valid_bs = 16
    fold = 0
 
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
 
    model = MX.to(device)
 
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_aug = albumentations.Compose(
        [   albumentations.CenterCrop(
                height=300,
                width=300,
            ),
         
            albumentations.Normalize(
                mean, 
                std, 
                max_pixel_value=255.0, 
                always_apply=True
            ),
            albumentations.ShiftScaleRotate(
                shift_limit=0.0625, 
                scale_limit=0.1, 
                rotate_limit=15
            ),
            albumentations.Flip(p=0.5)
        ]
    )
 
    valid_aug = albumentations.Compose(
        [
            albumentations.CenterCrop(
                height=300,
                width=300,
            ),
            albumentations.Normalize(
                mean, 
                std, 
                max_pixel_value=255.0,
                always_apply=True
            )
        ]
    )

    # df_train = df_train.iloc[:256]
    # df_valid = df_valid.iloc[:128]
 
    train_images = df_train.image_name.values.tolist()
    train_images = [
        os.path.join(training_data_path, i) for i in train_images
    ]
    train_targets = df_train.target.values
 
    valid_images = df_valid.image_name.values.tolist()
    valid_images = [
        os.path.join(training_data_path, i) for i in valid_images
    ]
    valid_targets = df_valid.target.values
 
    train_loader = ClassificationDataLoader(
        image_paths=train_images,
        targets=train_targets,
        resize=None,
        augmentations=train_aug,
    ).fetch(
        batch_size=train_bs, 
        drop_last=True, 
        num_workers=4, 
        shuffle=True, 
        tpu=True
    )
 
    valid_loader = ClassificationDataLoader(
        image_paths=valid_images,
        targets=valid_targets,
        resize=None,
        augmentations=valid_aug,
    ).fetch(
        batch_size=valid_bs, 
        drop_last=True,
        num_workers=2, 
        shuffle=False, 
        tpu=True
    )
 
        
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #     optimizer,
    #     patience=3,
    #     threshold=0.001,
    #     mode="min"
    # )

    scheduler = {'scheduler': torch.optim.lr_scheduler.OneCycleLR(
                                        optimizer,
                                        max_lr=1e-3,
                                        steps_per_epoch=int(len(train_loader)),
                                        epochs=NUM_EPOCHS,
                                        anneal_strategy="linear",
                                        final_div_factor = 30,
                                    ),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
 
    es = EarlyStopping(patience=15, mode="min", tpu=True)
    eng = Engine(model, optimizer, device=device, use_tpu=True, tpu_print=25)
 
 
 
    for epoch in range(epochs):
        train_loss = eng.train(train_loader)
        valid_loss, predictions = eng.evaluate(valid_loader, return_predictions=True)
        xm.master_print(f"Epoch = {epoch}, LOSS = {valid_loss}")

        #xm.master_print(valid_loss)
 
        
        es(valid_loss, model, model_path=f"drive/MyDrive/highly-probable/results/tp1/model_fold_{fold}.bin")
        if es.early_stop:
            xm.master_print("Early stopping")
            break
        gc.collect()

def validate():
    training_data_path = "imgs-300w/"
    df = load_df()
    device = xm.xla_device()
    epochs = NUM_EPOCHS
    train_bs = 32
    valid_bs = 16
    fold = 0
 
    # df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
 
    model = MX.to(device)
    model.load_state_dict(torch.load('drive/MyDrive/highly-probable/results/tp1/model_fold_0.bin'))

 
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    # train_aug = albumentations.Compose(
    #     [   albumentations.CenterCrop(
    #             height=300,
    #             width=300,
    #         ),
         
    #         albumentations.Normalize(
    #             mean, 
    #             std, 
    #             max_pixel_value=255.0, 
    #             always_apply=True
    #         ),
    #         albumentations.ShiftScaleRotate(
    #             shift_limit=0.0625, 
    #             scale_limit=0.1, 
    #             rotate_limit=15
    #         ),
    #         albumentations.Flip(p=0.5)
    #     ]
    # )
 
    valid_aug = albumentations.Compose(
        [
            albumentations.CenterCrop(
                height=300,
                width=300,
            ),
            albumentations.Normalize(
                mean, 
                std, 
                max_pixel_value=255.0,
                always_apply=True
            )
        ]
    )

    # df_train = df_train.iloc[:256]
    # df_valid = df_valid.iloc[:128]
 
    # train_images = df_train.image_name.values.tolist()
    # train_images = [
    #     os.path.join(training_data_path, i) for i in train_images
    # ]
    # train_targets = df_train.target.values
 
    valid_images = df_valid.image_name.values.tolist()
    valid_images = [
        os.path.join(training_data_path, i) for i in valid_images
    ]
    valid_targets = df_valid.target.values
 
    # train_loader = ClassificationDataLoader(
    #     image_paths=train_images,
    #     targets=train_targets,
    #     resize=None,
    #     augmentations=train_aug,
    # ).fetch(
    #     batch_size=train_bs, 
    #     drop_last=True, 
    #     num_workers=4, 
    #     shuffle=True, 
    #     tpu=True
    # )
 
    valid_loader = ClassificationDataLoader(
        image_paths=valid_images,
        targets=valid_targets,
        resize=None,
        augmentations=valid_aug,
    ).fetch(
        batch_size=valid_bs, 
        drop_last=True,
        num_workers=2, 
        shuffle=False, 
        tpu=True
    )
 
        
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    # scheduler = {'scheduler': torch.optim.lr_scheduler.OneCycleLR(
    #                                     optimizer,
    #                                     max_lr=1e-3,
    #                                     steps_per_epoch=int(len(train_loader)),
    #                                     epochs=NUM_EPOCHS,
    #                                     anneal_strategy="linear",
    #                                     final_div_factor = 30,
    #                                 ),
    #                     'name': 'learning_rate',
    #                     'interval':'step',
    #                     'frequency': 1}
 
    eng = Engine(model, optimizer, device=device, use_tpu=True, tpu_print=25)
    valid_accuracy, valid_loss, logits = eng.evaluate(valid_loader, return_predictions=True)
    # predictions = torch.argmax(torch.cat(logits), dim=1)
    # xm.master_print(logits)
    # xm.master_print(predictions)
    # xm.master_print(valid_targets)
    # valid_accuracy = metrics.accuracy_score(valid_targets, predictions)
    xm.master_print(f"LOSS = {valid_loss}, Accuracy = {valid_accuracy}")
    gc.collect()

        

In [ ]:
def _mp_fn(rank, flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    a = validate()

In [ ]:
FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

(277620, 3)
(277620, 3)
(277620, 3)
(277620, 3)
(277620, 3)
(277620, 3)
(277620, 3)
(277620, 3)
2021-02-26 08:37:32.495190: Batch 0 / 433, loss=1.5629571676254272
2021-02-26 08:37:41.672980: Batch 10 / 433, loss=0.7743805132128976
2021-02-26 08:37:55.196431: Batch 20 / 433, loss=0.7498890175705865
2021-02-26 08:38:05.578711: Batch 30 / 433, loss=0.7611670369102109
2021-02-26 08:38:19.842978: Batch 40 / 433, loss=0.7398029238712497
2021-02-26 08:38:29.539491: Batch 50 / 433, loss=0.7528594013522653
2021-02-26 08:38:40.492018: Batch 60 / 433, loss=0.7502749870057965
2021-02-26 08:38:51.672144: Batch 70 / 433, loss=0.7327653329137346
2021-02-26 08:39:05.342176: Batch 80 / 433, loss=0.7308958386197503
2021-02-26 08:39:15.580032: Batch 90 / 433, loss=0.738404645369603
2021-02-26 08:39:29.454421: Batch 100 / 433, loss=0.7408490564563487
2021-02-26 08:39:39.946443: Batch 110 / 433, loss=0.7389356502004572
2021-02-26 08:39:54.219307: Batch 120 / 433, loss=0.7316291177075757
2021-02-26 08:40:04

<function len>

In [ ]:

t1 = torch.Tensor([[ -4.0578,  -8.9004,  -8.1468,   0.7182,  -4.6377,  -1.4946,   9.7441,
          -1.7964,  -7.6763,  -9.0211,  -7.1281,  -1.6963,  -7.3100,  -8.5007,
          -0.4693,  -5.7752,  -2.2465,  -1.8670,  -8.1119,  -8.8775,  -8.6117,
           4.3674,   0.0494],
        [ -1.0636,  -4.8603,  -5.4210,  -0.1171,  -0.6040,   0.0831,   1.3921,
          -0.2309,  -4.1246,  -5.1452,  -5.6408,   1.3084,  -4.0589,  -5.8564,
           0.0450,  -2.5080,  -3.0302,  -2.4206,  -4.1956,  -5.3382,  -5.2597,
          -0.2480,   3.4566],
        [ -5.1050,  -6.6847,  -8.5644,  -4.0545,  -2.2365,  -1.1563,   1.3982,
           3.2515,  -8.2708,  -8.2989,  -8.3606,   0.7099,  -8.4915,  -7.7682,
          -4.2133,  -5.1981,  -0.4873,   3.9077,  -5.2250,  -8.8272,  -8.3565,
          -5.8237,   4.0996],
        [ -7.5464,  -6.5540,  -9.3766,  -0.8305,  -1.3218,  -6.1765,   4.2763,
           5.6337,  -9.6570,  -9.8515,  -9.7481,  -2.0424,  -9.0626,  -9.4960,
          -5.0519,  -4.9890,  -1.7645,  -2.7084,  -4.5378,  -9.8645, -10.0249,
          -4.2825,   6.8422],
        [ -4.2155,  -4.2437,  -7.7727,  -6.5253,  -1.6051,  -0.7649,  -2.6998,
           4.2403,  -7.1058,  -7.8559,  -7.7598,   1.6383,  -8.0333,  -5.7151,
          -2.3399,  -4.8864,   1.1154,   2.9296,  -4.3717,  -7.7189,  -7.7494,
          -6.5524,   5.3776],
        [ -5.2851,  -6.5032,  -9.2297,  -0.8373,  -1.7292,   2.8279,   0.8132,
           2.8236,  -7.7933,  -8.8084,  -9.1962,  -2.1960,  -8.0978,  -9.4600,
          -1.8449,  -5.5163,  -2.0939,  -3.3875,  -6.2630,  -9.4321,  -9.4624,
          -2.2305,   6.0902],
        [ -7.6281,  -9.3882, -11.0737,  -0.3564,   1.6851,  -3.9320,   2.9248,
           3.6319,  -8.9935, -10.3054, -10.8736,   3.7311,  -9.2857, -10.3548,
          -3.1580,  -6.6020,  -2.8683,  -1.8021,  -7.1932, -11.2721, -10.7029,
          -3.5078,   3.9736],
        [ -4.4476,  -6.5810,  -7.4404,  -3.1400,   7.6178,  -4.3544,  -1.0737,
           0.2824,  -4.4509,  -7.3714,  -6.9666,   6.4900,  -6.6098,  -4.5342,
          -1.6687,  -5.6462,   0.5463,  -1.7210,  -6.0462,  -7.6385,  -7.5721,
          -3.3123,  -0.3678],
        [ -4.1227,  -3.5180,  -4.2887,   0.8731,   3.6689,  -2.9151,   2.4010,
           1.8018,  -4.0304,  -4.1822,  -4.5264,  -0.3725,  -2.8682,  -4.3148,
          -2.1020,  -1.8784,  -1.8123,  -2.6236,  -2.6206,  -4.2139,  -4.5768,
          -2.0312,   2.8745],
        [ -2.0931,  -5.0425,  -5.2670,  -0.9187,   2.6347,  -0.7699,   1.8701,
           0.2768,  -6.4155,  -5.0374,  -4.8065,  -1.4019,  -5.5259,  -3.9943,
          -2.9691,  -3.6729,   1.4032,   3.0506,  -5.2287,  -6.0594,  -5.2305,
          -3.1503,   0.4525],
        [ -0.1351,  -6.0615,  -6.5711,  -1.5352,  -0.8255,  -1.4739,   0.6137,
          -2.1672,  -6.6945,  -8.0324,  -6.1045,   1.5999,  -6.5050,  -4.4149,
           0.0569,  -4.4863,   5.4237,  -1.4023,  -6.7684,  -7.3615,  -7.4003,
          -2.5453,   1.7930],
        [ -4.9942,  -9.9421,  -9.5870,   5.1570,  -7.3982,   2.6506,   9.5032,
          -1.5348,  -8.7071,  -8.7157,  -8.3847,  -3.9595,  -6.7236, -10.3208,
           0.5746,  -6.7496,  -5.1827,  -2.7215,  -9.2741, -10.3576,  -9.5665,
           3.4658,  -0.5893],
        [ -6.2687, -10.6241, -13.1800,  -5.1565,  -0.8236,  -5.0206,   1.1692,
           3.1116,  -9.1548, -12.7681, -13.3703,   7.8208, -11.8221, -11.0603,
           0.4807,  -7.4006,  -1.1081,  -4.0987,  -8.3184, -13.5922, -13.0280,
          -3.7718,   6.0637],
        [ -8.2072,  -9.3248, -11.7358,  -0.4060,  -0.0333,  -4.7856,   1.4977,
           4.5762, -10.6627, -11.2091, -11.5518,   2.2344, -10.0120, -10.4248,
          -5.6325,  -6.7217,  -0.4482,  -0.0416,  -7.1621, -11.7915, -11.2715,
          -7.8237,   6.5325],
        [ -2.9316,  -6.1250,  -7.9566,  -3.7964,  -1.9796,   1.0969,   0.8802,
           2.5595,  -4.4910,  -7.0841,  -8.3894,   1.8045,  -5.6817,  -7.7179,
           8.4842,  -4.2021,  -5.0693,  -7.5823,  -5.0515,  -8.0398,  -8.6495,
           0.2078,   1.6647],
        [ -1.4495,  -9.3525, -11.3788,  -2.0648,  -6.2584,   5.9604,  -1.7271,
          -1.4173, -10.4999, -10.2073,  -9.9474,   2.7338, -10.5014, -11.0423,
          -1.8098,  -8.3335,  -1.5104,   5.0161, -10.8629, -12.0137, -11.6607,
          -6.7809,   3.4897]])

t2 = torch.Tensor([[  1.6649,  -2.9340,  -6.6344,  -5.6007,  -4.3817,   3.7547,  -2.1206,
           4.2543,  -7.3630,  -7.0425,  -6.7711,  -4.0889,  -6.8821,  -7.2610,
           1.5472,  -4.8337,  -2.6937,  -2.4223,  -4.0609,  -6.9169,  -7.2993,
          -3.0769,   4.2746],
        [  4.4675,  -5.3455,  -8.2182,   1.5456,  -7.0646,   6.9733,   0.4629,
           1.1221,  -7.9156,  -7.6971,  -7.3783,  -4.5198,  -6.7829, -10.0985,
           2.6376,  -5.7024,  -6.9764,  -4.9502,  -6.6253,  -7.9667,  -7.6628,
          -1.3474,   1.4487],
        [ -5.6458,  -5.2420,  -7.0010,  -0.1460,   6.9970,  -3.5832,   2.1083,
           1.1481,  -6.3399,  -6.6563,  -6.6851,  -0.5752,  -5.5074,  -6.1518,
          -2.6929,  -3.8298,  -1.1626,  -1.6428,  -4.8282,  -6.8501,  -7.3331,
          -3.4557,   3.6197],
        [  0.3431,  -8.1226,  -8.9407,  -4.8710,   4.0835,  -4.1221,  -2.3645,
          -0.5717,  -6.4556,  -9.6048,  -8.9850,   8.7007,  -9.2128,  -6.9799,
           0.3366,  -6.7988,  -0.9254,  -2.7996,  -8.1522,  -9.7556,  -9.1448,
          -1.5034,  -0.1911],
        [ -5.8601,  -8.3552, -10.2026,  -2.1538,  -2.5582,   4.1812,   1.9199,
           0.7035,  -9.1477,  -9.2734,  -9.6069,  -2.5017,  -8.6097,  -8.4412,
           0.0670,  -6.0905,   1.5587,   1.4411,  -8.7537, -10.0778,  -9.9822,
          -3.6864,   3.1619],
        [ -5.0478,  -6.4017,  -9.6845,  -1.5056,  -4.6436,  -0.7071,   0.1037,
           9.3574, -10.7064,  -9.5382, -10.2290,  -3.1548,  -9.0742, -10.7330,
          -4.4986,  -5.7290,  -4.6972,  -3.0478,  -3.7863, -10.4164, -10.2749,
          -6.4920,   6.1659],
        [ -3.3377,  -7.9710,  -8.9440,  -0.8930,  -3.7937,   6.1331,   2.2229,
          -1.6892,  -8.0235,  -8.4267,  -8.0341,  -4.0309,  -7.4492,  -7.1280,
           2.5376,  -5.7231,   1.3994,   0.5770,  -9.5138,  -9.2044,  -8.9412,
          -1.0424,   0.3671],
        [ -6.6435,  -5.2929,  -9.1867,  -5.6656,   0.6932,  -3.8457,  -0.6334,
           7.3554,  -9.4301,  -8.7491,  -9.5693,  -0.3598,  -9.1305,  -7.3991,
          -3.9577,  -5.2902,   0.4465,   0.6632,  -3.4501,  -9.4808,  -9.5346,
          -6.8920,   4.7274],
        [ -5.4317,  -7.1617,  -8.4973,   1.8781,  -1.6755,   0.0758,   3.5512,
           9.0816, -10.0417,  -7.9978,  -8.6244,  -2.9238,  -7.4249,  -9.6715,
          -3.6050,  -5.7118,  -4.3633,  -3.0924,  -3.6113,  -9.1415,  -8.7007,
          -2.4588,   0.7310],
        [ -7.2998, -10.2992, -13.1481,  -0.8470,  -3.6656,   2.6011,   1.7698,
          -0.6456, -11.3088, -12.6157, -12.3345,  -0.4328, -10.8053, -12.2262,
          -3.5477,  -7.8529,  -1.8669,   2.0673, -10.8284, -13.1431, -12.7209,
          -5.7294,   7.6919],
        [ -2.2778,  -8.3753,  -9.3191,   0.2765,  -5.3698,   6.1114,   0.6886,
           0.9109,  -8.3857,  -8.0115,  -8.8855,  -1.0862,  -7.5735,  -9.4064,
          -2.6463,  -6.2093,  -3.4004,   2.2647,  -8.1020,  -9.5405,  -9.0627,
          -2.6494,   2.2780],
        [ -7.0492, -10.6521, -12.2627,  -1.4711,   3.4300,  -3.1939,   1.6283,
           0.6360,  -9.5827, -10.8267, -10.9385,   6.7426, -11.0197,  -8.7608,
          -3.2542,  -8.2067,   2.4477,   0.8747,  -9.7176, -12.5008, -12.0116,
          -3.6606,  -0.5525],
        [ -4.9558,  -7.7313, -10.6454,  -3.3818,  -3.6679,   4.6483,  -1.2995,
           1.7993,  -8.8944,  -9.0831,  -9.5370,   1.3152,  -9.2684,  -8.7975,
          -1.6107,  -6.7618,  -0.7401,   3.1490,  -8.3737, -10.4842, -10.0152,
          -5.7893,   3.2100],
        [ -3.0941,  -6.1330,  -7.1539,  -1.7411,   0.1286,   2.5211,   0.0275,
          -1.8025,  -6.2302,  -6.7477,  -6.1379,  -0.5557,  -6.6373,  -4.1707,
          -0.5339,  -4.5448,   3.8428,   3.1066,  -7.5896,  -7.2921,  -7.1417,
          -2.8434,   0.3889],
        [ -4.7475,  -9.2243,  -9.3524,   0.6677,  -3.3686,   5.7136,   0.9498,
          -1.1142,  -9.0059,  -7.9423,  -8.1285,  -2.9046,  -7.9405,  -7.4894,
          -2.4120,  -6.2783,   2.0715,   5.9911,  -9.8596,  -9.5603,  -9.1462,
          -3.8669,  -0.3228],
        [  0.1315,  -6.2035,  -7.3645,   0.4825,  -3.3153,   2.4951,   1.7093,
           0.0315,  -7.2340,  -7.3340,  -6.4412,   2.7079,  -7.6853,  -8.2445,
          -0.4751,  -5.2220,  -0.6158,  -0.7399,  -6.6951,  -8.2998,  -7.8399,
          -3.1746,   1.8055]])

In [ ]:
t1.shape

In [ ]:
asd = [t1, t2]

In [ ]:
asd = torch.cat(asd)

In [ ]:
torch.argmax(asd, dim=1).shape

In [ ]:
df[df.target=='']